In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

basket = pd.read_csv("Groceries_dataset.csv")
display(basket)


,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [3]:
#بتحول الداتا من bread => ['bread']
basket.itemDescription = basket.itemDescription.transform(lambda x: [x])
#بجمع البيانات حسب رقم العميل والتاريخ كل عمليه شراء فى نفس اليوم لنفس العميل تعتبر عمليه واحده 
basket = basket.groupby(['Member_number','Date']).sum()['itemDescription'].reset_index(drop=True)
# بتحول القوائم لبينارى  الاعمده هيا المنتجات والصفوف تمثل كل عملية شراء 
encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(basket).transform(basket), columns=encoder.columns_)
display(transactions)

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14961,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
frequent_itemsets = apriori(transactions, min_support= 6/len(basket), use_colnames=True, max_len = 3)
rules = association_rules(frequent_itemsets,num_itemsets= 9, metric="lift",  min_threshold = 2)
display(rules)
print("Rules identified: ", len(rules))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(soda),(artif. sweetener),0.097106,0.001938,0.000468,0.004818,2.485725,1.0,0.000280,1.002893,0.661986,0.004746,0.002885,0.123098
1,(artif. sweetener),(soda),0.001938,0.097106,0.000468,0.241379,2.485725,1.0,0.000280,1.190178,0.598864,0.004746,0.159790,0.123098
2,(brandy),(whole milk),0.002540,0.157923,0.000869,0.342105,2.166281,1.0,0.000468,1.279957,0.539750,0.005444,0.218724,0.173803
3,(whole milk),(brandy),0.157923,0.002540,0.000869,0.005501,2.166281,1.0,0.000468,1.002978,0.639347,0.005444,0.002969,0.173803
4,(butter milk),(long life bakery product),0.017577,0.017911,0.000668,0.038023,2.122893,1.0,0.000354,1.020907,0.538408,0.019194,0.020479,0.037668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,(yogurt),"(rolls/buns, whipped/sour cream)",0.085879,0.002941,0.000601,0.007004,2.381800,1.0,0.000349,1.004092,0.634653,0.006818,0.004075,0.105775
156,"(sausage, shopping bags)",(yogurt),0.001938,0.085879,0.000401,0.206897,2.409178,1.0,0.000235,1.152588,0.586056,0.004587,0.132387,0.105783
157,(yogurt),"(sausage, shopping bags)",0.085879,0.001938,0.000401,0.004669,2.409178,1.0,0.000235,1.002744,0.639872,0.004587,0.002736,0.105783
158,"(yogurt, whole milk)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,1.0,0.000797,1.082219,0.548014,0.020992,0.075973,0.078050


Rules identified:  160
